In [ ]:
import cv2
import numpy as np
import os
from google.colab.patches import cv2_imshow
from PIL import Image, ImageDraw, ImageFont
from math import floor

BASE_PATH = os.path.join('.', 'drive', 'My Drive', 'IZ*Net')
PROTOTXT_PATH = os.path.join(BASE_PATH, 'model_data', 'deploy.prototxt')
CAFFEMODEL_PATH = os.path.join(BASE_PATH, 'model_data', 'weights.caffemodel')
IMAGE_PATH = os.path.join(BASE_PATH, '480.jpg')
FONT_PATH = os.path.join(BASE_PATH, 'Roboto-Regular.ttf')

def drawImg(image, boundingBoxRow, boundingBoxCol, boundingBoxInfo):
  Font = ImageFont.truetype(FONT_PATH, 20)

  width, height = image.size

  Draw = ImageDraw.Draw(image)
  LABEL = "member"
  LABEL_SIZE = Draw.textsize(LABEL, Font)

  midX = (boundingBoxCol + boundingBoxInfo[0]) * (width / 10)
  midY = (boundingBoxRow + boundingBoxInfo[1]) * (height / 10)

  startX = midX - ((boundingBoxInfo[2] * width) / 2)
  endX = midX + ((boundingBoxInfo[2] * width) / 2)
  startY = midY - ((boundingBoxInfo[3] * height) / 2)
  endY = midY + ((boundingBoxInfo[3] * height) / 2)

  for i in range(10):
    Draw.rectangle([(startX + i, startY + i), (endX - i, endY - i)], outline = 'red')

  if startY - label_size[1] >= 0:
      text_origin = np.array([startX, startY - label_size[1]])
  else:
      text_origin = np.array([startX, startY + 1])
  
  Draw.rectangle([tuple(text_origin), tuple(text_origin + label_size)], fill='red')
  Draw.text(text_origin, label, fill=(0, 0, 0), font=Font)

# Read the model
print('Reading model ...')
model = cv2.dnn.readNetFromCaffe(PROTOTXT_PATH, CAFFEMODEL_PATH)
print('Model read ...')

image = cv2.imread(IMAGE_PATH)
(h, w) = image.shape[:2]

blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
model.setInput(blob)
detections = model.forward()

theImage = Image.open(IMAGE_PATH).convert('RGB')

for i in range(detections.shape[2]):
    # box= detections[0, 0, i, 3:7] * np.array([w, h, w, h])
    (startX, startY, endX, endY) = detections[0, 0, i, 3:7]

    midpointX = (startX + endX) / 2
    midpointY = (startY + endY) / 2

    boundingBoxRow = floor(midpointY * 10)
    boundingBoxCol = floor(midpointX * 10)

    percentBoundingBoxX = (midpointX * 10) - floor(midpointX * 10)
    percentBoundingBoxY = (midpointY * 10) - floor(midpointY * 10)

    percentageWidth = endX - startX
    percentageHeight = endY - startY
 
    confidence = detections[0, 0, i, 2]

    if (confidence > 0.5):
      drawImg(theImage, boundingBoxRow, boundingBoxCol, [percentBoundingBoxX, percentBoundingBoxY, percentageWidth, percentageHeight])

cv2_imshow(np.array(theImage)[...,::-1])

        # frame = image[startY:endY, startX:endX]
        # try:
        #     outfileName = "{}-{}{}".format(os.path.splitext(image_name)[0], i, os.path.splitext(image_name)[1])
        #     print("Saving ... {}".format(os.path.join(FACES_PATH, member, outfileName)))
        #     cv2.imwrite(os.path.join(FACES_PATH, member, outfileName), frame)
        # except Exception as e:
        #     print("Exception occured: {}".format(e))
        #     continue

In [ ]:
import cv2
import numpy as np
import os
from google.colab.patches import cv2_imshow
from PIL import Image, ImageDraw, ImageFont
from math import floor

BASE_PATH = os.path.join('.', 'drive', 'My Drive', 'IZ*Net')
PROTOTXT_PATH = os.path.join(BASE_PATH, 'model_data', 'deploy.prototxt')
CAFFEMODEL_PATH = os.path.join(BASE_PATH, 'model_data', 'weights.caffemodel')
IMAGE_PATH = os.path.join(BASE_PATH, '480.jpg')
FONT_PATH = os.path.join(BASE_PATH, 'Roboto-Regular.ttf')

# Read the model
print('Reading model ...')
model = cv2.dnn.readNetFromCaffe(PROTOTXT_PATH, CAFFEMODEL_PATH)
print('Model read ...')

image = cv2.imread(IMAGE_PATH)
(h, w) = image.shape[:2]

blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
model.setInput(blob)
detections = model.forward()

output = [[[] for i in range(10)] for j in range(10)]

for i in range(detections.shape[2]):
    (startX, startY, endX, endY) = detections[0, 0, i, 3:7]

    midpointX = (startX + endX) / 2
    midpointY = (startY + endY) / 2

    boundingBoxRow = floor(midpointY * 10)
    boundingBoxCol = floor(midpointX * 10)

    percentBoundingBoxX = (midpointX * 10) - floor(midpointX * 10)
    percentBoundingBoxY = (midpointY * 10) - floor(midpointY * 10)

    percentageWidth = endX - startX
    percentageHeight = endY - startY
 
    confidence = detections[0, 0, i, 2]

    if (confidence > 0.5):
      output[boundingBoxRow][boundingBoxCol] += [1, percentBoundingBoxX, percentBoundingBoxY, percentageWidth, percentageHeight]

for i in range(10):
  for j in range(10):
    [output[i][j].append(0) for _ in range(30 - len(output[i][j]))]

print(np.array(output).reshape((3000)))

Reading model ...
Model read ...
[0. 0. 0. ... 0. 0. 0.]
